In [509]:
import numpy as np

In [510]:
import sympy

In [511]:
perch_length = np.array(
    [8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 
     21.0, 21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 
     22.5, 22.7, 23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 
     27.3, 27.5, 27.5, 27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 
     36.5, 36.0, 37.0, 37.0, 39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 
     40.0, 42.0, 43.0, 43.0, 43.5, 44.0]
     )
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

In [512]:
class Dense:
  """
  완전 연결층의 연산을 하는 Dense 층, 가중치를 임의로 생성하거나, 입력받아 연산을 수행하고,
  그 값을 다음 층의 노드로 전달한다. (중간층, 결과층)
  """
  # 가중치 w
  w = []
  # 입력에 편향 벡터를 추가하기 위한 열이 추가된 데이터
  x_b = []

  def cal_dense(self, input, unit):
    """
    랜덤한 가중치를 생성하여 입력 값과 다음 층에 대한 내적 연산을 수행하는 함수
    args
      input : 입력 데이터
      unit : 은닉층 노드의 개수(다음층의 노드 개수)
    return 
      각 가중치 별 입력 데이터와의 연산 결과
    """
    # 입력 데이터에 편향 가중치를 위한 열, 1의 추가
    self.x_b = np.c_[input, np.ones((input.shape[0],1))]
    
    # 가중치 w의 생성
    self.w = np.random.rand(self.x_b.shape[0], unit)

    return input.T.dot(self.w)

  def cal_dense_weight(self, input, weight, unit):
    """
    가중치를 입력받아 입력값과의 내적 연산, 이 때 가중치는 편향 가중치가 마지막 열에 존재하는 형태
    args
      input : 입력 데이터
      weight : 가중치
      unit : 다음층의 노드 개수
    return
      각 가중치 별 입력 데이터와의 연산 결과
    """
    # 입력 데이터에 편향 가중치를 위한 열, 1의 추가
    self.x_b = np.c_[input, np.ones((input.shape[0],1))]
    self.w = weight

    return input.T.dot(self.w)

In [513]:
class activation:
  """
  각 활성화 함수의 정의
  """
  # 활성화 함수
  fx = 0

  # 활성화 함수의 미분ㄱ밧
  dfx = 0

  def step_fun(self, input, threshold = 0):
    """
    계단 함수, 임계값에 대한 입력을 받을 수 있다.
    Args:
      input : 입력값
      threshold : 임계값, 기본값 = 0
    Return:
      각 입력에 대한 계단함수 연산 결과
    """
    result = []

    for i in range(input.shape[1]):
      if(input[0][i] > threshold):
        result.append(1)
      else:
        result.append(0)    
    
    return np.array(result).reshape(1, -1)

  def sign_fun(self, input, threshold = 0):
    """
    부호 함수 
    입력이 0 미만일 경우 -1을 출력하고 0을 초과할 때 1을 출력, 0일 때 0을 출력
    임곗값을 입력받아 지정할 수 있다.
    Args:
      input : 입력값
      threshold : 임계값, 기본값 = 0
    Return:
      -1,0 or 1
    """
    result = []

    for i in range(input.shape[1]):
      if(input[0][i] > threshold):
        result.append(1)
      elif (input[0][i] == threshold):
        result.append(0)
      else:
        result.append(-1)
    
    return np.array(result).reshape(1, -1)

  
  def sigmoid_fun(self, input):
    """
    시그모이드 함수
    자연 상수의 지수 함수를 사용한 0~1 사이의 값 출력
    Args:
      x : 입력값
    Return:
      0~1 사이의 값
    """
    x = sympy.Symbol('x')
    e = sympy.Symbol('e')
    result = []

    fx = 1/(1+e**(-x))
    self.fx = fx

    dfx = sympy.diff(fx, x)
    self.dfx = dfx

    for i in range(input.shape[1]):
      result.append(fx.subs([(e,np.e), (x,input[0][i])]))

    return np.array(result).reshape(1, -1)

  def hyperbolic_tangent(self, input):
    """
    하이퍼볼릭 탄젠트 함수

    시그모이드 함수를 변형한 함수
    Args:
      input : 입력값
    Return:
      -1 ~ 1
    """
    x = sympy.Symbol('x')
    e = sympy.Symbol('e')
    result = []

    fx = 2*(1/(1+e**-x))-1
    self.fx = fx

    dfx = sympy.diff(fx, x)
    self.dfx = dfx

    for i in range(input.shape[1]):
      result.append(fx.subs([(e,np.e), (x,input[0][i])]))

    return np.array(result).reshape(1, -1)

  def relu(self, input, threshold = 0):
    """
    렐루 함수

    입력값이 0 이하면 0을 출력, 양수이면 그대로 그 값을 출력
    Args:
      input : 입력값
      threshold : 임계값, 기본값 = 0
    Return:
      0 ~ 
    """ 
    result = []

    for i in range(input.shape[1]):
      if(input[0][i] > threshold):
        result.append(input[0][i])
      else:
        result.append(0)

    return np.array(result).reshape(1, -1)

  def leaky_relu(self, input, threshold, a=0.01):
    """
    리키 렐루 함수

    입력값이 0 이하면 기울기 a를 곱한 값을 출력, 양수이면 그대로 그 값을 출력
    Args:
      input : 입력값
      threshold : 임계값
      a : 기울기(기본값 0.01)
    Return:
      연산 결과
    """
    result = []

    for i in range(input.shape[1]):
      if(input[0][i] > threshold):
        result.append(input[0][i])
      else:
        result.append(input[0][i]*a)

    return np.array(result).reshape(1, -1)

  def softmax_fun(self, input):
    """
    소프트 맥스 함수

    입력 리스트들에 대한 각 요소들의 비율, 확률을 계산
    오버플로를 예방하기 위한 최댓값을 빼주는 과정
    Args:
      input : 입력값
    Returen:
      연산 결과
    """
    sum = 0
    max_input = max(input[0])
    result = []

    for i in range(input.shape[1]):
      sum += np.e**(input[0][i] - max_input)

    for i in range(input.shape[1]):
      result.append(np.e**(input[0][i] - max_input) / sum)

    return np.array(result).reshape(1, -1)


In [514]:
class Mlp(Dense, activation):
  def a(self):
    print('a')

In [515]:
perch_length = perch_length.reshape(-1,1)

In [516]:
mlp = Mlp()

In [517]:
hidden_input = mlp.cal_dense(perch_length, 2)
hidden_input

array([[760.56011734, 819.66983917]])

In [518]:
hidden_input[0][1]

819.6698391742265

In [519]:
activation_step_result = mlp.step_fun(hidden_input, 0)
activation_step_result

array([[1, 1]])

In [520]:
activation_sign_result = mlp.sign_fun(hidden_input, 800)
activation_sign_result

array([[-1,  1]])

In [521]:
activation_sigmoid_result = mlp.sigmoid_fun(hidden_input)
activation_sigmoid_result

array([[1.00000000000000, 1.00000000000000]], dtype=object)

In [522]:
hyperbolic_tangent_result  = mlp.hyperbolic_tangent(hidden_input)
hyperbolic_tangent_result

array([[1.00000000000000, 1.00000000000000]], dtype=object)

In [523]:
relu_result = mlp.relu(hidden_input)
relu_result

array([[760.56011734, 819.66983917]])

In [524]:
leaky_relu_result = mlp.relu(hidden_input)
leaky_relu_result

array([[760.56011734, 819.66983917]])

In [525]:
softmax_result = mlp.softmax_fun(hidden_input)
softmax_result

array([[2.13291712e-26, 1.00000000e+00]])

In [526]:
mlp.fx

-1 + 2/(1 + e**(-x))